In [1]:
# Import the init_paths module
from init_paths import (
    PROJECT_ROOT,
    DATASETS_DIR,
    ANALYSIS_DIR,
    EXPERIMENTS_DIR,
    IMAGES_DIR,
    GEN_DATA_DIR,
    LOGS_DIR,
    get_project_path
)

Paths initialized successfully. You can now use PROJECT_ROOT, DATASETS_DIR, etc.


In [2]:
import pandas as pd
import sys
from pathlib import Path
import os
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
import pickle

In [3]:
from analysis.supervised import *
from analysis.models import * 

## Load data

In [9]:
publication = ["public_1", "public_2"][1]

folder_path = DATASETS_DIR / f"prompt_activations/{publication}/"
files = os.listdir(folder_path)

models_list = list(models.keys())
model = models_list[1]

for file in files:
    
    if file.endswith(".pkl") and "rated" in file and model in file: 
      print(file)
      path = folder_path / file

      df = pd.read_pickle(path)




_meta-llamaLlama-3.1-8B-Instruct_prompt_res_activations_2102202513:10:37_rated.pkl


In [21]:
model_name, num_layers, rating_column, hf_ident, input_size = get_model_details(path)

layer= num_layers - 4

# layer = 17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wildJailBreak" if "wildJailBreak" in str(path) else "wei_dataset"

df["rating_binary"] = df[rating_column].apply(lambda x: 0 if x == "No" else 1)
df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant", "gcg", "autodan"])]
df["jailbreak_prompt_name"].value_counts()
#shuffle and balance dataset
df = df.sample(frac=1).reset_index(drop=True)

df = balance_dataset(df, "rating_binary")
df["rating_binary"].value_counts()

rating_binary
1    3633
0    3633
Name: count, dtype: int64

In [22]:
def save_data_and_labels(df, layers, filepath):
    """
    Prepare data by extracting features, melting the dataframe, and splitting into train/val sets.
    Args:
        df (pd.DataFrame): Input DataFrame containing the data
        layers (list): List of layers to extract features from
        filepath (str): Path to save the tensors and CSV
    """
    # Extract features for each layer
    for layer in layers:
        df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)
    
    # Store original DataFrame before melting for CSV export
    df_for_csv = df.copy()
    
    # Melt the DataFrame
    melted_df = pd.melt(
        df,
        id_vars=["jailbreak_prompt_name", "rating_binary"],
        value_vars=[f'layer_{layer}' for layer in layers],
        var_name="layer",
        value_name="features"
    )
    
    # Prepare labels and data
    labels = melted_df["rating_binary"].to_list()
    data = melted_df["features"].to_list()
    
    # Convert to tensors
    data_tensor = torch.tensor(np.stack(data), dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    
    # Save tensors
    dataset_dict = {
        'data': data_tensor,
        'labels': labels_tensor
    }
    torch.save(dataset_dict, filepath)
    
    # Save CSV without tensor columns
    tensor_cols = ['prompt_res_activations'] + [f'layer_{i}' for i in layers]
    df_for_csv_no_tensors = df_for_csv.drop(tensor_cols, axis=1)
    df_for_csv_no_tensors.to_csv(filepath.replace(".pt", ".csv"), index=True)

filepath = f"{path}_tensors.pt"
layers = range(0, num_layers)
save_data_and_labels(df, layers, filepath)


In [23]:
# Test if your tensor saving worked

# Load the saved tensor data
loaded_data = torch.load(filepath)

# Extract data and labels from the loaded dictionary
data_tensor = loaded_data['data']
labels_tensor = loaded_data['labels']

# Print shapes to verify the loading
print(f"Data tensor shape: {data_tensor.shape}")
print(f"Labels tensor shape: {labels_tensor.shape}")

/tmp/ipykernel_3227610/31292468.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(filepath)


Data tensor shape: torch.Size([261576, 4096])
Labels tensor shape: torch.Size([261576])


In [24]:
# Load the saved CSV data
csv_filepath = filepath.replace(".pt", ".csv")
df_loaded = pd.read_csv(csv_filepath, index_col=0)

# Print info to verify loading
print("\nLoaded CSV info:")
print(df_loaded.info())




Loaded CSV info:
<class 'pandas.core.frame.DataFrame'>
Index: 7266 entries, 0 to 7265
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   prompt_name            7266 non-null   object
 1   jailbreak_prompt_name  7266 non-null   object
 2   jailbreak_prompt_text  7266 non-null   object
 3   original_prompt_text   7266 non-null   object
 4   output_text            7265 non-null   object
 5   rating_output_text     7266 non-null   object
 6   rating_binary          7266 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 454.1+ KB
None
